In [131]:
import pandas as pd
import datetime as dt
import simplejson as json
from escapejson import escapejson

# SQLAlchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
sample_size = 0.01 # 1% sample size

from sqlalchemy import Column, Integer, String, Float, DateTime, null, ForeignKey

In [132]:
class TaxiTrip(Base):
    __tablename__ = 'taxi_trip'
    trip_id = Column(Integer, primary_key=True, autoincrement=True)
    vendor_id = Column(Integer, nullable=True)
    pickup_time = Column(DateTime, nullable=True) 
    dropoff_time = Column(DateTime, nullable=True) 
    passenger_count = Column(Float, nullable=True)
    trip_distance = Column(Float, nullable=True)
    pickup_loc_id = Column(Integer, nullable=True)
    dropoff_loc_id = Column(Integer, nullable=True)
    total_amount = Column(Float, nullable=True)
    payment_type = Column(Integer, nullable=True)
    taxi_type = Column(Integer, nullable=True)

In [133]:
class ForHireTrip(Base):
    __tablename__ = 'for_hire_trip'
    trip_id = Column(Integer, primary_key=True, autoincrement=True)
    company = Column(String(10), nullable=True)
    pickup_time = Column(DateTime, nullable=True) 
    dropoff_time = Column(DateTime, nullable=True) 
    pickup_loc_id = Column(Integer, nullable=True)
    dropoff_loc_id = Column(Integer, nullable=True)
    shared_flag = Column(Integer, nullable=True)

In [134]:
class Zone(Base):
    __tablename__ = 'zone'
    location_id = Column(Integer, primary_key=True, autoincrement=False)
    borough = Column(String(100), nullable=True)
    zone = Column(String(100), nullable=True)
    service_zone = Column(String(100), nullable=True)
    #"LocationID","Borough","Zone","service_zone"

In [135]:
# Path to sqlite
database_path = "database/ny_taxi_data.sqlite"

# Create an engine that can get to the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

# Use this to clear out the db
Base.metadata.drop_all(engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

from sqlalchemy.orm import Session
session = Session(bind=engine)

In [136]:
#import the Zones Data csv file into dataframe
file_movie = "raw-data/zones/taxi_zone_lookup.csv"

#columns to import from csv file .. 
col_list = ["LocationID","Borough","Zone","service_zone"]
zones_df = pd.read_csv(file_movie,usecols=col_list, low_memory=False)
zones_df.head(5)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [137]:
#read each row from the dataframe and store in the TaxiTrip table
print("Starting Zones Migration")
for index, row in zones_df.iterrows():
    zone = Zone(location_id=row["LocationID"], borough=row["Borough"], zone=row["Zone"], 
                service_zone=row["service_zone"])
    session.add(zone)

session.commit() #save the session
print("Finished Zones Migration")

Starting Zones Migration
Finished Zones Migration


In [138]:
#Yellow Taxi .. taxi_type = 1
#import the Yellow Taxi Data csv file into dataframe
file_movie = "raw-data/taxis/yellow_tripdata_2019-12.csv"

#columns to import from csv file .. 
col_list = ["VendorID","tpep_pickup_datetime","tpep_dropoff_datetime", "passenger_count","trip_distance","PULocationID","DOLocationID","payment_type","total_amount"]
yellow_taxi_df = pd.read_csv(file_movie,usecols=col_list, low_memory=False)
yellow_taxi_df = yellow_taxi_df.sample(frac=sample_size)
yellow_taxi_df.head(5)
#yellow_taxi_df.dtypes

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,total_amount
4879629,2.0,2019-12-20 15:27:29,2019-12-20 15:33:07,1.0,0.65,237,237,1.0,10.56
1275834,2.0,2019-12-06 13:12:41,2019-12-06 13:25:09,1.0,0.90,230,161,2.0,11.80
6801135,1.0,2019-12-31 19:05:51,2019-12-31 19:17:44,4.0,2.60,234,48,1.0,17.02
5983281,1.0,2019-12-26 21:52:06,2019-12-26 21:57:11,1.0,0.60,163,162,1.0,10.56
2287503,1.0,2019-12-10 17:04:09,2019-12-10 17:17:05,1.0,1.30,162,170,1.0,16.55


In [139]:
#read each row from the dataframe and store in the TaxiTrip table
print("Starting Yellow Taxi Migration")
count = 0
for index, row in yellow_taxi_df.iterrows():
    pickup_dt = null()
    dropoff_dt = null()
    if not pd.isna(row["tpep_pickup_datetime"]): #check to see if the date is not null or not na
        try: #try/except to avoid error due to date converion from string 2019-12-01 00:05:27
            pickup_dt = dt.datetime.strptime(row["tpep_pickup_datetime"], '%Y-%m-%d %H:%M:%S')
        #break
        except ValueError:
            pickup_dt = null()
            print(f'{row["tpep_pickup_datetime"]}, {index}')
            
    if not pd.isna(row["tpep_dropoff_datetime"]): #check to see if the date is not null or not na
        try: #try/except to avoid error due to date converion from string 2019-12-01 00:05:27
            dropoff_dt = dt.datetime.strptime(row["tpep_dropoff_datetime"], '%Y-%m-%d %H:%M:%S')
        #break
        except ValueError:
            dropoff_dt = null()
            print(f'{row["tpep_dropoff_datetime"]}, {index}')
            
    #creating the TaxiTrip object and adding it to the db session
    trip = TaxiTrip(vendor_id=row["VendorID"], pickup_time=pickup_dt,
                    dropoff_time=dropoff_dt,passenger_count=row["passenger_count"], 
                    trip_distance=row["trip_distance"], pickup_loc_id=row["PULocationID"], 
                    dropoff_loc_id=row["DOLocationID"], total_amount=row["total_amount"],
                    payment_type=row["payment_type"], taxi_type=1)
    session.add(trip)
    count = count + 1
    if count % 10000 == 0:
        print(f'{count}', end="\r")
        session.commit()

if session.is_modified:
    session.commit() #save the session
print("Finished Yellow Taxi Migration")

Starting Yellow Taxi Migration
Finished Yellow Taxi Migration


In [140]:
#Green Taxi .. taxi_type = 2
#import the Green Taxi Data csv file into dataframe
file_movie = "raw-data/taxis/green_tripdata_2019-12.csv"

#columns to import from csv file .. 
col_list = ["VendorID","lpep_pickup_datetime","lpep_dropoff_datetime", "passenger_count","trip_distance","PULocationID","DOLocationID","payment_type","total_amount"]
green_taxi_df = pd.read_csv(file_movie,usecols=col_list, low_memory=False)
green_taxi_df = green_taxi_df.sample(frac=sample_size)
green_taxi_df.head(5)
#green_taxi_df.dtypes

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,total_amount,payment_type
291653,2.0,2019-12-24 20:51:34,2019-12-24 21:00:34,260,129,1.0,1.44,8.80,2.0
175985,2.0,2019-12-15 03:45:15,2019-12-15 04:11:55,255,41,5.0,8.14,36.66,1.0
298594,2.0,2019-12-25 21:04:52,2019-12-25 21:12:22,75,74,1.0,1.12,8.30,2.0
233634,2.0,2019-12-19 20:33:02,2019-12-19 20:41:08,129,129,1.0,0.99,8.30,1.0
273387,2.0,2019-12-23 09:06:08,2019-12-23 09:15:28,75,168,2.0,2.10,11.76,1.0


In [141]:
#read each row from the dataframe and store in the TaxiTrip table
print("Starting Green Taxi Migration")
count = 0
for index, row in green_taxi_df.iterrows():
    pickup_dt = null()
    dropoff_dt = null()
    if not pd.isna(row["lpep_pickup_datetime"]): #check to see if the date is not null or not na
        try: #try/except to avoid error due to date converion from string 2019-12-01 00:05:27
            pickup_dt = dt.datetime.strptime(row["lpep_pickup_datetime"], '%Y-%m-%d %H:%M:%S')
        #break
        except ValueError:
            pickup_dt = null()
            print(f'{row["tpep_pickup_datetime"]}, {index}')
            
    if not pd.isna(row["lpep_dropoff_datetime"]): #check to see if the date is not null or not na
        try: #try/except to avoid error due to date converion from string 2019-12-01 00:05:27
            dropoff_dt = dt.datetime.strptime(row["lpep_dropoff_datetime"], '%Y-%m-%d %H:%M:%S')
        #break
        except ValueError:
            dropoff_dt = null()
            print(f'{row["tpep_dropoff_datetime"]}, {index}')
            
    #creating the TaxiTrip object and adding it to the db session
    trip = TaxiTrip(vendor_id=row["VendorID"], pickup_time=pickup_dt,
                    dropoff_time=dropoff_dt,passenger_count=row["passenger_count"], 
                    trip_distance=row["trip_distance"], pickup_loc_id=row["PULocationID"], 
                    dropoff_loc_id=row["DOLocationID"], total_amount=row["total_amount"],
                    payment_type=row["payment_type"], taxi_type=2)
    session.add(trip)
    count = count + 1
    if count % 10000 == 0:
        print(f'{count}', end="\r")
        session.commit()

if session.is_modified:
    session.commit() #save the session
print("Finished Green Taxi Migration")

Starting Green Taxi Migration
Finished Green Taxi Migration


In [142]:
#import the For Hire Taxi Data csv file into dataframe
file_movie = "raw-data/taxis/fhvhv_tripdata_2019-12.csv"

#columns to import from csv file .. 
col_list = ["hvfhs_license_num","pickup_datetime","dropoff_datetime", "PULocationID","DOLocationID","SR_Flag"]
for_hire_df = pd.read_csv(file_movie,usecols=col_list, low_memory=False)
for_hire_df = for_hire_df.sample(frac=sample_size)
for_hire_df.head(5)
#for_hire_df.dtypes

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
11645843,HV0003,2019-12-16 06:35:53,2019-12-16 06:53:13,89,61,NaN
10722624,HV0004,2019-12-14 22:34:09,2019-12-14 22:55:26,249,224,NaN
19569123,HV0003,2019-12-27 22:05:50,2019-12-27 22:18:25,116,168,NaN
132356,HV0003,2019-12-01 05:17:41,2019-12-01 05:32:38,37,129,NaN
13697770,HV0003,2019-12-18 22:13:43,2019-12-18 22:42:40,144,265,NaN


In [143]:
#read each row from the dataframe and store in the TaxiTrip table
print("Starting For Hire Migration")
count = 0
for index, row in for_hire_df.iterrows():
    pickup_dt = null()
    dropoff_dt = null()
    if not pd.isna(row["pickup_datetime"]): #check to see if the date is not null or not na
        try: #try/except to avoid error due to date converion from string 2019-12-01 00:05:27
            pickup_dt = dt.datetime.strptime(row["pickup_datetime"], '%Y-%m-%d %H:%M:%S')
        #break
        except ValueError:
            pickup_dt = null()
            print(f'{row["pickup_datetime"]}, {index}')
            
    if not pd.isna(row["dropoff_datetime"]): #check to see if the date is not null or not na
        try: #try/except to avoid error due to date converion from string 2019-12-01 00:05:27
            dropoff_dt = dt.datetime.strptime(row["dropoff_datetime"], '%Y-%m-%d %H:%M:%S')
        #break
        except ValueError:
            dropoff_dt = null()
            print(f'{row["dropoff_datetime"]}, {index}')
    
    company = null()
    if row["hvfhs_license_num"] == "HV0002":
        company = "Juno"
    elif row["hvfhs_license_num"] == "HV0003":
        company = "Uber"
    elif row["hvfhs_license_num"] == "HV0004":
        company = "Via"
    elif row["hvfhs_license_num"] == "HV0005":
        company = "Lyft"
    
    
    #creating the TaxiTrip object and adding it to the db session
    trip = ForHireTrip(company=company, pickup_time=pickup_dt,dropoff_time=dropoff_dt,
                       pickup_loc_id=row["PULocationID"],dropoff_loc_id=row["DOLocationID"],
                       shared_flag=row["SR_Flag"])
    session.add(trip)
    count = count + 1
    if count % 10000 == 0:
        print(f'{count}', end="\r")
        session.commit()

if session.is_modified:
    session.commit() #save the session
print("Finished For Hire Migration")

Starting For Hire Migration
Finished For Hire Migration
